# TO DO:

- error with ancillary features ---> NO CLUE!
    - de error die ik krijg komt niet altijd voor. Enkel bij bepaalde dataframes
    - Onderzoek de dataframe door te vergroten tot error op duikt. Dat weet je welke rij de error veroorzaakt en kan je mss oplossen
    
    
- filter inputdata so that at least 100 vandals -> make dataset more balanced!

- new feature: Identify large edit -> to protect vectorizer -> assume vandal!




- change threshold for predicting a certain class!
  
- make an ensemble




- EXTRA: nog een aantal extra ancillary features toevoegen!


# Part I: Preprocessing the Document Edits

## Step 0: Import, Initialization and Loading

IDEA: load all the part files into a single dataframe

In [1]:
import pandas as pd
import numpy as np
import emoji
import re as re
import difflib
import shutil
import sys
import os

import profanity_check as pc
from fuzzywuzzy import fuzz

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)
        
# from pyspark import SparkContext
# SparkContext.setSystemProperty('spark.executor.memory', '2g')

from pyspark.ml import Pipeline
        
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler,IndexToString

from pyspark.sql.types import IntegerType, StringType, ArrayType, FloatType
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit, monotonically_increasing_id

from pyspark.sql.types import StructType
from pyspark.sql.types import StructField

from pyspark.ml.classification import NaiveBayes, NaiveBayesModel, RandomForestClassifier
from pyspark.ml.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import MultilabelMetrics
from pyspark.mllib.util import MLUtils

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from nltk.stem.snowball import *

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import multilabel_confusion_matrix



//anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be 

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

### Loading the data into a single dataframe

The idea is to load all the saved partfiles into a single dataframe. Next this dataframe can be used to train the model

In [4]:
def load_rdd(base_directory):
    # Get all the directory names of the saved myoutput folders
    foldernames = os.listdir(base_directory)
    
    # Create list of full directorie names
    full_directories = []
    
    for i in range(len(foldernames)):
        
        if foldernames[i] == '.DS_Store':
            continue
        
        directory_temp = base_directory + "/" + foldernames[i]
        full_directories.append(directory_temp)
    
    print("Number of directories included: ", len(full_directories))
    
    df = spark.read.format('json').load(path=full_directories)
    return df

# base_directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Cleaned/myoutput-1586797640000/part-00003'
# base_directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Cleaned/myoutput-1586797640000'
# base_directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Data_Limited'
base_directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Cleaned'

df = load_rdd(base_directory)
print(len(df.toPandas()))
df.show()
    
df_selection = df.toPandas().head(10)

## To run faster we only take a selection of the data:
df_selection = sqlContext.createDataFrame(df_selection)


Number of directories included:  1120
2703
+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             comment| label|           name_user|            text_new|            text_old|          title_page|            url_page|
+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       (→‎Etymology)|vandal|2601:442:4580:559...|{{redirect2|Masje...|{{redirect2|Masje...|              Mosque|//en.wikipedia.or...|
|→‎Comedy:#blackAF...|  safe|          Thornstrom|{{about||Netflix ...|{{about||Netflix ...|List of original ...|//en.wikipedia.or...|
|→‎2010s:Minor spe...|  safe|       BorderRegions|{{short descripti...|{{short descripti...|List of attacks r...|//en.wikipedia.or...|
|Minor edit in 'Re...|  safe|         Legofan2006|{{distinguish|Ron...|{{distinguish|Ron...|   Cristiano Ronaldo|//en.wikipedia.or...|
|           

### Checking the frequency of each label

In [5]:
df_selection.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------+-----+
| label|count|
+------+-----+
|  safe|    8|
|unsafe|    1|
|vandal|    1|
+------+-----+



## Step 1a: Tokenization & Normalization

The regexTokenizer is used because of its extra functionality compared to the standard Tokenizer built into spark. Also useful is that the tokens are normalized (decapitalized). 

In [6]:
def tokenize(dataframe):
    df = dataframe
    
    rt_old = RegexTokenizer(inputCol="text_old", outputCol="words_old", toLowercase=True, pattern=("\\W"))
    countTokens = udf(lambda words: len(words), IntegerType())

    # regexTokenized_old = rt_old.transform(df)
    # df_step1a = regexTokenized_old.withColumn("tokens_old", countTokens(col("words_old")))

    regexTokenized_old = rt_old.transform(df)
    df = regexTokenized_old.withColumn("tokens_old", countTokens(col("words_old")))

    #########################################################################################

    rt_new = RegexTokenizer(inputCol="text_new", outputCol="words_new", toLowercase=True, pattern=("\\W"))
    # regexTokenized_new = rt_new.transform(df_step1a)
    # df_step1b = regexTokenized_new.withColumn("tokens_new", countTokens(col("words_new")))

    regexTokenized_new = rt_new.transform(df)
    df = regexTokenized_new.withColumn("tokens_new", countTokens(col("words_new")))

    # df_step1b.show(truncate=False)
    
    print("Step 1a Done")
    return df


## Step 1b: Delta Generator

In this crucial step the difference between input and output text is determined. The difference is found using the unified_diff function accesible in through the difflib python library. The function takes two lists of strings as inputs and computes the deleted and inserted (replaced) words. This difference is used to later classify the text edit.

In [7]:
def text_difference(text_old,text_new):
#     text_old = df_step1b.select("words_old").collect()[0][0]
#     text_new = df_step1b.select("words_new").collect()[0][0]

    # print(text_old)

    new_words = []
    deleted_words = []

    for line in difflib.unified_diff(text_old, text_new, fromfile='before.txt', tofile='after.txt'):
    #     sys.stdout.write(line)

        if "-" in line and " " not in line:
            new_line = line.replace("-", "")
            deleted_words.append(new_line)
        elif "+" in line and " " not in line:
            new_line = line.replace("+", "")
            new_words.append(new_line)

    #     print(line)


    # print("Deleted words: ", deleted_words)
    # print("Inserted words: ", new_words)

    edited_words = deleted_words + new_words
    
    if len(edited_words) == 0:
        edited_words = ["empty"]
    
    print(edited_words)
    
    return edited_words

# text_old = df_step1b.select("words_old").collect()[0][0]
# text_new = df_step1b.select("words_new").collect()[0][0]
# edited_words = text_difference(text_old,text_new)


### User Defined Function

This code calculated the difference between the input and output text. This is accomplished by defining a UDF and a seperate function arrayUdf(). The udf is called on two columns *'words_old'* and *'words_new'*. Next a lambda function is defined to iterate over each row of the two input columns. Within the udf is refered to another function arrayUdf() which requires two inputs: the two tokenized lists of words which will be used to compute the difference. The arrayUdf() function acts as an itermediary to call on a different function: text_difference(). The text_difference() function uses the unified_diff generator from the difflib package to return the deltas between two lists of strings.

Through experimentation with the unified_diff generator, we found that it was much easier to first tokenize the input and output text and then compute the difference between the two tokenized lists of words. This in contrast to passing the two texts (*'text_old'* and *'text_new'*) of the rdd's as input directly and then tokenizing this *'difference_text'*. Although the latter method might create less computational overhead due to less tokenization, the former method proves to be much more reliable to determine which words have been deleted and which words are new.

In [8]:
def diff_text(dataframe):
    df = dataframe
    
    def arrayUdf(text_old,text_new):
        edited_words = text_difference(text_old,text_new)
        return edited_words

    # countTokens = udf(lambda words: len(words), IntegerType())
    callArrayUdf = udf(lambda row: arrayUdf(row[0],row[1]), ArrayType(StringType()))

    spark.udf.register("callArrayUdf",callArrayUdf)
    #calling udf function


    # df_step1c = df_step1b.withColumn("diff_text", callArrayUdf(struct('words_old','words_new')))

    df = df.withColumn("diff_text", callArrayUdf(struct('words_old','words_new')))

    # df_step1c.select("diff_text").show(truncate=False)
    
    print("Step 1b Done")
    return df

In [9]:
# Cross validate the change by checking the difference in number of tokens created

# tokens_old = df_step1b.select("tokens_old").collect()[0][0]
# tokens_new = df_step1b.select("tokens_new").collect()[0][0]

# diff_tokens = tokens_new - tokens_old
# print("The difference in number of tokens for input and output text = ", diff_tokens)

## Step 2: Stop Word Removal

In [10]:
def stop_words_removal(dataframe):

    df = dataframe
    
    locale = sc._jvm.java.util.Locale
    locale.setDefault(locale.forLanguageTag("en-US"))

    stopwords = StopWordsRemover.loadDefaultStopWords("english")
    extra_stopwords = ["http","https","ref","www","com","org","url","web"]
    stopwords = stopwords + extra_stopwords
    # print(stopwords)

    remover = StopWordsRemover(inputCol="diff_text", outputCol="words_clean",stopWords=stopwords)
    stopwords = remover.getStopWords()


    # df_step2 = remover.transform(df_step1c)

    # df_step2.select("words_clean").show(truncate=False)

    df = remover.transform(df)

    # (inputCol="words", outputCol="filtered",stopWords=StopWordsRemover.loadDefaultStopWords("english"))
    # remover.transform(df_tokenized).show(truncate=False)
    
    print("Step 2 Done")
    return df

## Step 3: Stemming

The chosen algorithm for stemming is the snowball stemming algorithm (a variant of the Porter algorithm). The snowball stemmer was chosen because it is slightly more aggresive at stemming the tokenized words than the standard Porter algorithm while still being less aggresive than the Lancaster algorithm. It is a nice 'middle ground' between the two stemming variants.

In [11]:
def stemming(dataframe):
    
    df = dataframe
    
    # stemmer = SnowballStemmer('english')
    stemmer = PorterStemmer()

    stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))

    # df_step3 = df_step2.withColumn("words_stemmed", stemmer_udf("words_clean"))

    df = df.withColumn("words_stemmed", stemmer_udf("words_clean"))

    # df_step3.select("words_stemmed").show(truncate=False)
    
    print("Step 3 Done")
    return df

## Step 4: Feature Vectorization (TF-IDF)

In [12]:
def vectorization(dataframe):
    
    df = dataframe
    
    tf = HashingTF(inputCol="words_stemmed", outputCol="tf")#, numFeatures=20)

    # df_step4a = tf.transform(df_step3)
    df = tf.transform(df)


    idf = IDF(inputCol="tf", outputCol="tf_idf")
    # idfModel = idf.fit(df_step4a)
    # df_step4b = idfModel.transform(df_step4a)

    idfModel = idf.fit(df)
    df = idfModel.transform(df)

    # df_step4a.show(truncate=False)
    # df_step4b.select("words_stemmed","tf_idf").show(truncate=False)
    
    print("Step 4 Done")
    return df

## Step 5: String Indexer

In this final step the labels (*Safe, Unsafe and Vandal*) are encoded to label indices. The most frequent label gets index 0 while the least frequent label gets the last index depending on the number of indices. In this case the least frequent label gets index 2.

In our data 0 corresponds to *Safe*, 1 corresponds to *Unsafe*, and 2 corresponds to *Vandal*.

In [13]:
def string_indexer(dataframe):
    
    df = dataframe
    
    label_indexer = StringIndexer(inputCol = "label", outputCol = "label_index")
    
#     indexToLabel = label_indexer.labels
    
    # df_step5a = label_indexer.fit(df_step4b).transform(df_step4b)
    # df_step5b = df_step5a.select("tf_idf","label_index")

    #  # Renaming the columns
    # df_final = df_step5b.withColumnRenamed("tf_idf","features")
    # df_final = df_final.withColumnRenamed("label_index","label")

    df = label_indexer.fit(df).transform(df)
    

    # # Renaming the columns
    df_label = df.select("tf_idf","label_index")
    df_label = df_label.withColumnRenamed("label_index","actual_label")

    # df_final.show()
    
    print("Step 5 Done")
    return df , df_label #, indexToLabel

## Step 6: Assembly of the preprocessing steps

In [14]:
def preprocessing(dataframe):
    
    df = dataframe
    
    df = tokenize(df)
    df = diff_text(df)
    df = stop_words_removal(df)
    df = stemming(df)
    df = vectorization(df)
    df , df_label = string_indexer(df)
    
    return df,df_label


df_final , df_label = preprocessing(df_selection)

    

Step 1a Done
Step 1b Done
Step 2 Done
Step 3 Done
Step 4 Done
Step 5 Done


# Part II: Ancillary Features



In [15]:
# old_text = ["car", "train", "boat"]
# new_text = ["car", "train", "boat", "biiike", "pl4n3"]
# diff_text = ["biiike", "pl4n3", "pl4n3"]

# word = 'biiike'

In [16]:
def maxRepeat(diff_text):
    
    h = len(diff_text)
    count = 0
    
    for i in range(0, h):
        l = len(diff_text[i])
        
        #Find the maximum repeating character
                   
        for j in range(0, l):
            cur_count = 1
            for k in range(j + 1, l):
                if(diff_text[i][j] != diff_text[i][k]):
                    break
                cur_count += 1
                                
                #update result if required
                if cur_count > count:
                    count = cur_count
                            
    return count

# print(maxRepeat(diff_text))



In [17]:
#empty revision
##checks if edit is is empty or non empty

def empty(text_list):
    if len(text_list) == 0 or text_list[0] == "empty":
        empty = 1
    else:
        empty = 0

    return empty

In [18]:
#ratio between old text and new text; if  > 1 new text is longer than old text
#i would consider it vandal if there is a significant deviation from 1

def size_ratio(old_text_list, new_text_list):
    len_old_text = len(old_text_list)
    len_new_text = len(new_text_list)
    
    if len_old_text == 0:
        ratio = 0.0
    else:
        ratio = round(len_new_text / len_old_text,3)
    
    return ratio

In [19]:
#counts the alphanumberic strings in the diffrence list eg (dkfdj125kd,...) the strings with numbers and letters
## Since these strings are likely to be vandal
## Absolute count or ratio better?

def alphanumeric_count(difference_list):
    alpha_num = 0
    for element in difference_list:
        if element.isdigit():
            continue
        elif element.isalpha():
            continue
        else:
            alpha_num += 1
    
    return alpha_num

In [20]:
#ratio of vulgar words in the edit

def vulgar(old_text_list,difference_list):
    
    vulgar_list_edit = pc.predict(difference_list)
    vulgar_list_old = pc.predict(old_text_list)
    count1 = 0
    count2 = 0
    
    for i in vulgar_list_edit:
        count1 += i
        
    for k in vulgar_list_old:
        count2 += k
        
    ratio1 = count1 #/ len(difference_list)
    
    if count2 == 0:
        ratio2 = 0.0
    
    else:
        ratio2 = round(count1 / count2,3)
        
    return ratio2

In [21]:
#Gives a similarity metric between original and new text
#How less similar the more suspicous

def similarity(old_text_list, new_text_list):
    old = ''.join(old_text_list)
    new = ''.join(new_text_list)
    ratio = round(fuzz.token_set_ratio(old, new)/100,3)
    
    return ratio

In [22]:
def compute_ancillary_features(dataframe):
    df = dataframe
    
    ## UDF for computing longest repeated character
    cafUdf1 = udf(lambda row: maxRepeat(row[2]), IntegerType())
    spark.udf.register("cafUdf1", cafUdf1)
    df = df.withColumn("longest_repeated_char", cafUdf1(struct('words_old','words_new','diff_text')))
    
    ## UDF for checking if edit is empty or not
    cafUdf2 = udf(lambda row: empty(row[2]), IntegerType())
    spark.udf.register("cafUdf2", cafUdf2)
    df = df.withColumn("empty_edit", cafUdf2(struct('words_old','words_new','diff_text')))
    
    ## UDF to determine size ratio between input and output text
    cafUdf3 = udf(lambda row: size_ratio(row[0],row[1]), FloatType())
    spark.udf.register("cafUdf3", cafUdf3)
    df = df.withColumn("size_ratio", cafUdf3(struct('words_old','words_new','diff_text')))
    
    ## UDF to determine number of alphanumeric words in an edit
    cafUdf4 = udf(lambda row: alphanumeric_count(row[2]), IntegerType())
    spark.udf.register("cafUdf4", cafUdf4)
    df = df.withColumn("alpha_count", cafUdf4(struct('words_old','words_new','diff_text')))
    
    ## UDF to determine the ratio of vulgar words in the text
    cafUdf5 = udf(lambda row: vulgar(row[0],row[2]), FloatType())
    spark.udf.register("cafUdf5", cafUdf5)
    df = df.withColumn("vulgar_ratio", cafUdf5(struct('words_old','words_new','diff_text')))
    
    ## UDF to determine the ratio of vulgar words in the text
    cafUdf6 = udf(lambda row: similarity(row[0],row[1]), FloatType())
    spark.udf.register("cafUdf6", cafUdf6)
    df = df.withColumn("similarity", cafUdf6(struct('words_old','words_new','diff_text')))
    
    print("Ancillary Features Done")
    return df

In [23]:
df_ancillary = compute_ancillary_features(df_final)

# pd_df_ancillary = df_ancillary.toPandas()
# pd_df_ancillary = pd_df_ancillary[pd_df_ancillary['empty_edit'] == 0]
# df_ancillary = sqlContext.createDataFrame(pd_df_ancillary)

df_ancillary = df_ancillary.select('diff_text','tf_idf','longest_repeated_char','empty_edit','size_ratio','alpha_count',\
                                   'vulgar_ratio','similarity','label_index')
df_ancillary.show()


Ancillary Features Done


Py4JJavaError: An error occurred while calling o506.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 13.0 failed 1 times, most recent failure: Lost task 3.0 in stage 13.0 (TID 4317, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:90)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:89)
	at scala.collection.Iterator$$anon$12.nextCur(Ite

# Part III: Building Models

## Multinomial Naive Bayes Classifier

In [ ]:
# rdd_final = df_final.rdd
(training_data_nb, test_data_nb) = df_label.randomSplit([0.7, 0.3], seed = 42)


In [ ]:
# Show the frequency of each label in the test set
# test_data.groupBy("actual_label") \
#     .count() \
#     .orderBy(col("count").desc()) \
#     .show()

In [ ]:
# from pyspark.ml.classification import LogisticRegression

# lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
# lrModel = lr.fit(training_data)
# predictions = lrModel.transform(test_data)

In [ ]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",featuresCol='tf_idf', labelCol='actual_label')
model_nb = nb.fit(training_data_nb)
predictions_nb = model_nb.transform(test_data_nb)

# # Convert indexed labels back to original labels.
# label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel",labels=indexToLabel)
# predictions_nb = label_converter.fit(predictions_nb).transform(predictions_nb)

In [ ]:
# predictions.filter(predictions['prediction'] == 1).select('label','probability','prediction').show(truncate=False)

In [ ]:
# predictions.filter(predictions['prediction'] == 2).select('label','probability','prediction').show(truncate=False)

In [ ]:
# predictions.filter(predictions['label'] == 1).select('label','probability','prediction').show(truncate=False)

In [ ]:
# predictions.filter(predictions['label'] == 2).select('label','probability','prediction').show(truncate=False)

### Saving the Model
Use pickle package to save a model to a file and load a model from a file.


In [ ]:
# output_dir = r'/Users/Simon/Documents/GitHub/adana_task3'
# shutil.rmtree(output_dir, ignore_errors=True)
# model_nb.save(sc, output_dir)
# sameModel = nb.load(sc, output_dir)

## Random Forest Classisfier

In [ ]:
assembler = VectorAssembler(inputCols=['longest_repeated_char','empty_edit','size_ratio','alpha_count','vulgar_ratio','similarity'],\
                            outputCol='features')

df_ancillary = assembler.transform(df_ancillary)
df_ancillary_vector = df_ancillary.select('features','label_index')
df_ancillary_vector.show()

In [ ]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df_ancillary_vector.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="label_index", featuresCol="features", numTrees=10)

# Convert indexed labels back to original labels.
# labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
#                                labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
# pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model_rf = rf.fit(trainingData)

# Make predictions.
predictions_rf = model_rf.transform(testData)
predictions_rf.show()

# Part IV - A: Evaluation of the Naive Bayes Model

## Accuracy


In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol='actual_label')
accuracy_nb = evaluator.evaluate(predictions_nb)
print(accuracy_nb)
# Result = 0.7410821256831497
# Result = 0.8053634745632435
# Result = 0.8838408650684599
# Result = 0.911560434636496

## Precision and Recall

For the precision and recall measures we fall back on Sklearn packages because their implementation is much more straight forward as compared to the mllib packages. We are mostly interested in the unsafe and vandal isntances!

**Binary Classification:**

- Precision = TruePositives / (TruePositives + FalsePositives)
- Recall = TruePositives / (TruePositives + FalseNegatives)

**Multilabel Classification:**

In an imbalanced classification problem with more than two classes, precision is calculated as the sum of true positives across all classes divided by the sum of true positives and false positives across all classes.

- Precision = Sum c in C TruePositives_c / Sum c in C (TruePositives_c + FalsePositives_c)


In an imbalanced classification problem with more than two classes, recall is calculated as the sum of true positives across all classes divided by the sum of true positives and false negatives across all classes.

- Recall = Sum c in C TruePositives_c / Sum c in C (TruePositives_c + FalseNegatives_c)


In [ ]:
pd_predictions_nb = predictions_nb.toPandas()
# print(pd_predictions_nb[pd_predictions_nb.prediction == 2.0].count())


y_true = pd_predictions_nb['actual_label'].to_list()
y_pred = pd_predictions_nb['prediction'].to_list()

# precision_recall_fscore_support(y_true, y_pred, average='micro')
# precision_recall_fscore_support(y_true, y_pred, average='macro')
precision_recall_fscore_support(y_true, y_pred, average='weighted')
# precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0.0, 1.0, 2.0])
# pd_predictions_nb.head()

## Confusion Matrices and Manual Calculations

In [ ]:
cm_arr = multilabel_confusion_matrix(y_true, y_pred,labels=[0.0, 1.0, 2.0])
cm_arr_safe = cm_arr[0,:,:]
cm_arr_unsafe = cm_arr[1,:,:]
cm_arr_vandal = cm_arr[2,:,:]

cm_safe = pd.DataFrame({'Pred_Other': cm_arr_safe[:,0], 'Pred_0': cm_arr_safe[:,1]})
cm_safe = cm_safe.rename(index={0: 'Act_Other', 1: 'Act_0'})

cm_unsafe = pd.DataFrame({'Pred_Other': cm_arr_unsafe[:,0], 'Pred_1': cm_arr_unsafe[:,1]})
cm_unsafe = cm_unsafe.rename(index={0: 'Act_Other', 1: 'Act_1'})

cm_vandal = pd.DataFrame({'Pred_Other': cm_arr_vandal[:,0], 'Pred_2': cm_arr_vandal[:,1]})
cm_vandal = cm_vandal.rename(index={0: 'Act_Other', 1: 'Act_2'})

cm_vandal.head()

In [ ]:
precision_safe = cm_arr_safe[1,1]/(cm_arr_safe[1,1] + cm_arr_safe[0,1])
precision_unsafe = cm_arr_unsafe[1,1]/(cm_arr_unsafe[1,1] + cm_arr_unsafe[0,1])
precision_vandal = cm_arr_vandal[1,1]/(cm_arr_vandal[1,1] + cm_arr_vandal[0,1])

recall_safe = cm_arr_safe[1,1]/(cm_arr_safe[1,1] + cm_arr_safe[1,0])
recall_unsafe = cm_arr_unsafe[1,1]/(cm_arr_unsafe[1,1] + cm_arr_unsafe[1,0])
recall_vandal = cm_arr_vandal[1,1]/(cm_arr_vandal[1,1] + cm_arr_vandal[1,0])

print("Precision for Label 0: ", precision_safe)
print("Precision for Label 1: ", precision_unsafe)
print("Precision for Label 2: ", precision_vandal)

print("Recall for Label 0: ", recall_safe)
print("Recall for Label 1: ", recall_unsafe)
print("Recall for Label 2: ", recall_vandal)

tp_tot = cm_arr_safe[1,1]+cm_arr_unsafe[1,1]+cm_arr_vandal[1,1]
fp_tot = cm_arr_safe[0,1]+cm_arr_unsafe[0,1]+cm_arr_vandal[0,1]
fn_tot = cm_arr_safe[1,0]+cm_arr_unsafe[1,0]+cm_arr_vandal[1,0]

precision_tot = (tp_tot / (tp_tot + fp_tot))
recall_tot = ( tp_tot / (tp_tot + fn_tot ))

print("Total Precision: ", precision_tot)
print("Total Recall: ", recall_tot)
              

# Part IV - B: Evaluation of the Random Forest Model

In [ ]:
evaluator2 = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol='label_index')
accuracy_rf = evaluator2.evaluate(predictions_rf)
print(accuracy_rf)

In [ ]:
pd_predictions_rf = predictions_rf.toPandas()
# print(pd_predictions_nb[pd_predictions_nb.prediction == 2.0].count())


y_true = pd_predictions_rf['label_index'].to_list()
y_pred = pd_predictions_rf['prediction'].to_list()

# precision_recall_fscore_support(y_true, y_pred, average='micro')
# precision_recall_fscore_support(y_true, y_pred, average='macro')
precision_recall_fscore_support(y_true, y_pred, average='weighted')
# precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0.0, 1.0, 2.0])
# pd_predictions_nb.head()

# Part V: Running the Models in a Streaming Setup

In [ ]:
globals()['models_loaded'] = False

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("===================== %s =====================" % str(time))
    
    ## Convert to data frame
    df_pred = spark.read.json(rdd)
    print("Incoming Dataframe: ")
    df_pred.show()

    
    ## Preprocessing the incoming dataframe 
    df_pred_final , df_pred_label = preprocessing(df_pred)
    print("Preprocessed Dataframe: ")
    df_pred_final.show()
    
    
    ## Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model_nb'] = model_nb 
        globals()['models_loaded'] = True #Update the control to notify model is loaded
        
    # Predict using the loaded model: 
    df_result = globals()['my_model_nb'].transform(df_pred_label)
    print("Predicted Result: ")
    df_result.show()

    

In [ ]:
# ssc = StreamingContext(sc, 10)

In [ ]:
# lines = ssc.socketTextStream("seppe.net", 7778)
# lines.foreachRDD(process)

In [ ]:
# ssc_t = StreamingThread(ssc)
# ssc_t.start()

In [ ]:
# ssc_t.stop()